In [3]:
from ml.data import process_data
from ml.model import train_model, compute_model_metrics, inference
from sklearn.model_selection import train_test_split
from joblib import dump

# Add the necessary imports for the starter code.
import os
import sys
import pandas as pd

file_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.insert(0, file_dir)

In [9]:
data = pd.read_csv(file_dir + '/../data/clean_census.csv')

In [15]:
data.columns

Index(['age', 'workclass', 'fnlgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'salary'],
      dtype='object')

In [16]:
data.head(5)

,age,workclass,fnlgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [17]:
train, test = train_test_split(data, test_size=0.20)

cat_features = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
]

In [18]:
X_train, y_train, encoder, lb = process_data(
    train, categorical_features=cat_features, label="salary", training=True
)

In [19]:
X_test, y_test, encoder, lb = process_data(
    test, categorical_features=cat_features, label="salary", training=False, encoder=encoder, lb=lb
)

In [20]:
rf_model = train_model(X_train, y_train)

In [22]:
# from sklearn.utils import estimator_checks

# estimator_checks.check_estimator(rf_model)
rf_model.__class__.__name__

'RandomForestClassifier'

In [25]:
y= [1, 1, 0, 0]
predicts =  [0, 1,1, 0]
precision, recall, fbeta = compute_model_metrics(y, predicts)
print(precision, recall, fbeta)

0.5 0.5 0.5


In [26]:
train, test = train_test_split(data, test_size=0.20)

X, y, _, _ = process_data(
    train, categorical_features=cat_features, label="salary", training=True
)
assert len(X) == len(y)

In [27]:
train, test = train_test_split(data, test_size=0.20)
from sklearn.ensemble import RandomForestClassifier
X, y, _, _ = process_data(
    train, categorical_features=cat_features, label="salary", training=True
)
model = train_model(X, y)
# Check if this is a classification model
assert isinstance(model, RandomForestClassifier)

In [28]:
y = [1, 1, 0, 0]
predicts = [0, 1, 1, 0]
precision, recall, fbeta = compute_model_metrics(y, predicts)
print(precision, recall, fbeta)
assert abs(precision - 0.5) < 0.01 and abs(recall -
                                            0.5) < 0.01 and abs(fbeta - 0.5) < 0.01

0.5 0.5 0.5


In [ ]:
X, y, _, _ = process_data(
        data, categorical_features=cat_features, label="salary", training=True
    )
model = train_model(X, y)
pred = inference(model, X)
# Check if pred.shape is similar to y.shape
assert y.shape == pred.shape